In [1]:
import logging
import os
import sys
import re

import matplotlib.pyplot as plt

from tqdm import tqdm
from typing import List, Callable, NoReturn, NewType, Any
import dataclasses
from datasets import load_metric, load_from_disk, Dataset, DatasetDict

from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer
from transformers import (
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

In [2]:
sys.path.append('../')

In [3]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

from utils_qa import postprocess_qa_predictions, check_no_error
from trainer_qa import QuestionAnsweringTrainer
from retrieval import SparseRetrieval

from arguments import (
    ModelArguments,
    DataTrainingArguments,
)

## Raw Data

In [4]:
datasets = load_from_disk('/opt/ml/data/train_dataset')

In [5]:
train_data = datasets['train']
train_size = len(train_data)

## Check Context 

In [18]:
target_data = train_data['context'][2000:]

In [115]:
def remove_newline(txt) :
    txt = txt.replace(r'\n*' , '')
    txt = txt.replace(r'\n#' , '')
    txt = txt.replace(r'\n' , '')
    return txt

## Tokenizer

In [28]:
model_args = ModelArguments

config = AutoConfig.from_pretrained(
    model_args.config_name 
    if model_args.config_name is not None
    else model_args.model_name_or_path,
)

# -- Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name
    if model_args.tokenizer_name is not None
    else model_args.model_name_or_path,
    # 'use_fast' argument를 True로 설정할 경우 rust로 구현된 tokenizer를 사용할 수 있습니다.
    # False로 설정할 경우 python으로 구현된 tokenizer를 사용할 수 있으며,
    # rust version이 비교적 속도가 빠릅니다.
    use_fast=True,
)
 

In [163]:
print(remove_newline(target_data[5]))

아기스 3세는 선왕 아르키다모스 3세의 아들이며, 후대의 왕 에우다미다스 1세의 동생이다. 그는 카이로네이아 전투가 있던 날에 왕위에 올랐다.기원전 333년 그는 에게 해에서 페르시아 제국 아케메네스 왕조 지휘관 파르나바조스, 아우토프라다테스와 회담을 갖고 마케도니아 왕국의 지배에 대항하여 스파르타의 반란을 일으키면 원조를 받기로 했다. 같은 해 〈이수스 전투〉 이후 아기스는 계획을 실행에 옮겼다. 우선 동생 아게실라오스를 크레타로 보내 기원전 331년에 스파르타를 비롯한 대 마케도니아 동맹에 참여했다 그리스 여러 도시 국가들은 마케도니아에 대한 트라키아의 반란을 틈타 대 마케도니아 전쟁을 선언했다.(그리스-마케도니아 전쟁). 그러나 그리스 도시 국가들이 굳건하게 뭉친 것은 아니어서, 아테나이 등이 불참했다. 아기스는 스파르타 군과 이수스 전투에서 페르시아와 싸운 그리스인 용병 8000명, 그리고 동맹군을 더해 보병 20,000명, 기병 2,000기를 밑돌지 않는 병력을 모아 아기스 스스로 최고 사령관이 되었다. 그는 코라고스 휘하의 마케도니아 군을 펠로폰네소스에서 물리치고 결정적인 승리를 거두었다.다른 동맹군(엘리스, 아카이아, 아르카디아)의 합류로 아기스는 메갈로폴리스를 포위했다. 반면 비운의 알렉산더 3세로부터 그리스를 맡았던 마케도니아 장군 안티파트로스는 트라키아에서 반란을 정리하고, 즉시 40,000명 이상의 대군을 이끌고 메갈로폴리스를 구원하러 왔다. 그리고 양군은 〈가우가멜라 전투〉가 벌어진 것과 같은 시기에 회전을 치뤘다. (메갈로폴리스 전투) 격전이 계속되었지만, 마케도니아 군의 병력수에 밀린 아기스 군이 패배를 당했다. 아기스는 부상을 당했으면서도 계속 싸우다가 전사했다. 이 패배가 결정적이 되어 스파르타가 마케도니아를 상대로 일으킨 반란은 스파르타의 패배라는 결과로 끝났다.아기스는 자식을 남기지 않았기 때문에, 동생 에우다미다스가 다음 왕위를 계승했다.


In [158]:
print(tokenizer.tokenize("야권 단일화가 합의되어 진보신당의 김석준, 민주노동당 민병렬 후보와 경선 끝에 야권단일 후보로 선출되었다. 6.2 지방선거에서 44.57% 득표를 하였지만 석패했다."))

['야권', '단일', '##화가', '합의', '##되', '##어', '진보신', '##당', '##의', '김석', '##준', ',', '민주', '##노동', '##당', '민병', '##렬', '후보', '##와', '경선', '끝', '##에', '야권', '##단', '##일', '후보', '##로', '선출', '##되', '##었', '##다', '.', '6', '.', '2', '지방', '##선거', '##에서', '44', '.', '57', '%', '득표', '##를', '하', '##였', '##지만', '석', '##패', '##했', '##다', '.']


### CheckPoint

  1. index : 5 --> 〈가우가멜라 전투〉가 벌어진 것과 같은 시기에 회전을 치뤘다. (메갈로폴리스 전투) 
  2. index : 6 --> 2심에서 징역10년을 선고받은 김종원(29세 여성) 피고인에 대해 원심을 파기하고 징역15년을 선고했다.
  3. index : 8 --> 기원후 3세기 중엽 심하바르만 1세(275년 ~ 300년)
  4. index : 11 -->  《폴아웃 4》에 대한 평론가의 평가는 세 플랫폼 모두에서 호의적이다.
  5. index : 18 --> 갈릴레오 호가 탐사하던 때 이후 폭발한 화산 여럿의 사진을 찍었다2014년 현재 목성과 그 주변의 계를 탐사할 계획 2개가 예정되어 있다.
  6. index : 20 --> 쇠고기 등 적색육에서는 평균적으로 0.1 ~ 2.0%의 수치가 검출된다.
  7. index : 22 --> 《남극 활동 및 환경 보호에 관한 법률》(약칭 《남극 활동법》)에 의해 출입이 제한된다.
  8. index : 29 --> 정복자 로베르 기스카르(1015-1085)가 이끄는 군대가 로마에 입성한후 약탈을 자행하였다.
  9. index : 32 --> 더 많아 지는것 및 술을 줄이거나 끊었을 때 나타나는 불안, 불면, 설사, 환청, 환시, 간질발작과 같은 증상등이 나타나는데 이런 증상을 없애기 위해 술을 또 마시는것등이다.
  10. index : 37 --> 사회심리학자 지바 쿤다(Ziva Kunda)는 인지적 메카니즘과 동기 부여 메커니즘을 결합하여 편향을 만드는 것은 동기적 측면이지만 편향의 규모를 결정하는 것은 인지적 과정이라고 주장하였다198
  11. index : 38 -->  ① 카르텔 : 유럽 연합 및 유럽 경제 지역 내에 있어서 담합이나 과점에 대한 규제이다. ② 독점 : 기업의 시장에 있어서 우월적 지위의 남용을 방지하는 것이다. 유럽 연합의 기능에 관한 조약 제102조에 규정되어 있다. ③ 합병 : 유럽연합 및 유럽 경제 지역 내에 있어서 어느 일정한 매출액을 가진 기업이 관여하는 합병,
  12. index : 39 --> 그가 시인으로서 명성을 얻기 시작한 것은 예카테리나 대제에게 바치는 송시 ＜펠리사＞(1782)를 발표하면서부터다.
  13. index : 47 --> YMS-14형으로 설정이 바뀌었다.만화 《솔로몬의 악몽》(《가라!가라! 우리들의 V건담!!》: 고토부키 츠카사著 수록)에서는
  14. index : 

### Direction
  1. Stopwords 제거
    1. Retrieval 할 때 -> TF-IDF Vector 구할 때
    2. Training - 
